# Import Relevant Libraries and Load in Merged Datasets

In [1]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler,MaxAbsScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
import numpy as np
from xgboost import XGBClassifier

In [2]:
df = pd.read_csv('data/merged_data.csv')

/Users/jamiecarnevale/miniconda3/envs/learn-env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (10,21,30,63,97,99,118,119,120,122,126,127,128,131,133,136,137,139,149) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


# Data Cleaning

In [3]:
df.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 731907 entries, 0 to 731906
Data columns (total 150 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   Unnamed: 0                     731907 non-null  int64  
 1   PERSON_ID                      731907 non-null  object 
 2   PERSON_TYPE                    731907 non-null  object 
 3   CRASH_RECORD_ID                731907 non-null  object 
 4   RD_NO_x                        727570 non-null  object 
 5   VEHICLE_ID_x                   725151 non-null  float64
 6   CRASH_DATE_x                   731907 non-null  object 
 7   SEAT_NO                        60155 non-null   float64
 8   CITY                           460748 non-null  object 
 9   STATE                          459075 non-null  object 
 10  ZIPCODE                        422795 non-null  object 
 11  SEX                            727658 non-null  object 
 12  AGE                          

In [4]:
df.iloc[:,100:120].head()

,TOWED_BY,TOWED_TO,AREA_00_I,AREA_01_I,AREA_02_I,AREA_03_I,AREA_04_I,AREA_05_I,AREA_06_I,AREA_07_I,AREA_08_I,AREA_09_I,AREA_10_I,AREA_11_I,AREA_12_I,AREA_99_I,FIRST_CONTACT_POINT,CMV_ID,USDOT_NO,CCMC_NO
0,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FRONT,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SIDE-RIGHT-FRONT,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,REAR,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,FRONT-LEFT,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,UNKNOWN,NaN,NaN,NaN


### let's drop all columns that have less than 20k nonnull values

In [5]:
df.columns[df.notna().sum() < 20000]

Index(['EMS_RUN_NO', 'PEDPEDAL_ACTION', 'PEDPEDAL_VISIBILITY',
       'PEDPEDAL_LOCATION', 'BAC_RESULT VALUE', 'CELL_PHONE_USE',
       'PHOTOS_TAKEN_I', 'STATEMENTS_TAKEN_I', 'DOORING_I', 'WORK_ZONE_I',
       'WORK_ZONE_TYPE', 'WORKERS_PRESENT_I', 'CMRC_VEH_I', 'FIRE_I',
       'EXCEED_SPEED_LIMIT_I', 'CMV_ID', 'USDOT_NO', 'CCMC_NO', 'ILCC_NO',
       'COMMERCIAL_SRC', 'GVWR', 'CARRIER_NAME', 'CARRIER_STATE',
       'CARRIER_CITY', 'HAZMAT_PLACARDS_I', 'HAZMAT_NAME', 'UN_NO',
       'HAZMAT_PRESENT_I', 'HAZMAT_REPORT_I', 'HAZMAT_REPORT_NO',
       'MCS_REPORT_I', 'MCS_REPORT_NO', 'HAZMAT_VIO_CAUSE_CRASH_I',
       'MCS_VIO_CAUSE_CRASH_I', 'IDOT_PERMIT_NO', 'WIDE_LOAD_I',
       'TRAILER1_WIDTH', 'TRAILER2_WIDTH', 'TRAILER1_LENGTH',
       'TRAILER2_LENGTH', 'TOTAL_VEHICLE_LENGTH', 'AXLE_CNT', 'VEHICLE_CONFIG',
       'CARGO_BODY_TYPE', 'LOAD_TYPE', 'HAZMAT_OUT_OF_SERVICE_I',
       'MCS_OUT_OF_SERVICE_I', 'HAZMAT_CLASS'],
      dtype='object')

In [6]:
colstodrop = df.columns[df.notna().sum() < 200000]

In [7]:
df1 = df.drop(colstodrop, axis = 1)

In [8]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 731907 entries, 0 to 731906
Data columns (total 78 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   Unnamed: 0                     731907 non-null  int64  
 1   PERSON_ID                      731907 non-null  object 
 2   PERSON_TYPE                    731907 non-null  object 
 3   CRASH_RECORD_ID                731907 non-null  object 
 4   RD_NO_x                        727570 non-null  object 
 5   VEHICLE_ID_x                   725151 non-null  float64
 6   CRASH_DATE_x                   731907 non-null  object 
 7   CITY                           460748 non-null  object 
 8   STATE                          459075 non-null  object 
 9   ZIPCODE                        422795 non-null  object 
 10  SEX                            727658 non-null  object 
 11  AGE                            434077 non-null  float64
 12  DRIVERS_LICENSE_STATE         

In [9]:
df1.head()

,Unnamed: 0,PERSON_ID,PERSON_TYPE,CRASH_RECORD_ID,RD_NO_x,VEHICLE_ID_x,CRASH_DATE_x,CITY,STATE,ZIPCODE,...,LIC_PLATE_STATE,VEHICLE_YEAR,VEHICLE_DEFECT,VEHICLE_TYPE,VEHICLE_USE,TRAVEL_DIRECTION,MANEUVER,OCCUPANT_CNT,AREA_01_I,FIRST_CONTACT_POINT
0,0,O749947,DRIVER,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,JC451435,834816.0,09/28/2019 03:30:00 AM,CHICAGO,IL,60651,...,IL,2016.0,UNKNOWN,PASSENGER,PERSONAL,N,STRAIGHT AHEAD,1.0,Y,FRONT
1,1,O871921,DRIVER,af84fb5c8d996fcd3aefd36593c3a02e6e7509eeb27568...,JD208731,827212.0,04/13/2020 10:50:00 PM,CHICAGO,IL,60620,...,IL,NaN,NONE,PASSENGER,PERSONAL,W,STRAIGHT AHEAD,1.0,NaN,SIDE-RIGHT-FRONT
2,2,O10018,DRIVER,71162af7bf22799b776547132ebf134b5b438dcf3dac6b...,HY484534,9579.0,11/01/2015 05:00:00 AM,NaN,NaN,NaN,...,XX,NaN,NONE,PASSENGER,PERSONAL,E,BACKING,1.0,NaN,REAR
3,3,O10038,DRIVER,c21c476e2ccc41af550b5d858d22aaac4ffc88745a1700...,HY484750,9598.0,11/01/2015 08:00:00 AM,NaN,NaN,NaN,...,IL,2012.0,NONE,PASSENGER,NOT IN USE,N,PARKED,0.0,NaN,FRONT-LEFT
4,4,O10039,DRIVER,eb390a4c8e114c69488f5fb8a097fe629f5a92fd528cf4...,HY484778,9600.0,11/01/2015 10:15:00 AM,NaN,NaN,NaN,...,XX,NaN,UNKNOWN,UNKNOWN/NA,UNKNOWN/NA,UNKNOWN,UNKNOWN/NA,1.0,NaN,UNKNOWN


In [10]:
df1['SAFETY_EQUIPMENT'].value_counts()

USAGE UNKNOWN                                 440625
SAFETY BELT USED                              265755
NONE PRESENT                                   14629
SAFETY BELT NOT USED                            3329
HELMET NOT USED                                 2672
CHILD RESTRAINT USED                            1124
BICYCLE HELMET (PEDACYCLIST INVOLVED ONLY)       737
CHILD RESTRAINT - FORWARD FACING                 504
DOT COMPLIANT MOTORCYCLE HELMET                  477
HELMET USED                                      418
CHILD RESTRAINT - REAR FACING                    207
CHILD RESTRAINT - TYPE UNKNOWN                   205
NOT DOT COMPLIANT MOTORCYCLE HELMET              121
BOOSTER SEAT                                     110
SHOULD/LAP BELT USED IMPROPERLY                   93
CHILD RESTRAINT NOT USED                          83
WHEELCHAIR                                        31
CHILD RESTRAINT USED IMPROPERLY                   17
STRETCHER                                     

In [11]:
df1['MOST_SEVERE_INJURY'].value_counts()

NO INDICATION OF INJURY     632576
NONINCAPACITATING INJURY     56080
REPORTED, NOT EVIDENT        29971
INCAPACITATING INJURY        12470
FATAL                          799
Name: MOST_SEVERE_INJURY, dtype: int64

In [12]:
df1['VEHICLE_DEFECT'].value_counts()

UNKNOWN             398828
NONE                305305
OTHER                 4822
BRAKES                3524
TIRES                  632
STEERING               597
WHEELS                 329
SUSPENSION             222
ENGINE/MOTOR           129
FUEL SYSTEM            117
LIGHTS                  59
WINDOWS                 52
CARGO                   35
SIGNALS                 22
RESTRAINT SYSTEM        17
TRAILER COUPLING        15
EXHAUST                 11
Name: VEHICLE_DEFECT, dtype: int64

In [13]:
df1['PRIM_CONTRIBUTORY_CAUSE'].value_counts()

UNABLE TO DETERMINE                                                                 282207
FAILING TO YIELD RIGHT-OF-WAY                                                        80260
FOLLOWING TOO CLOSELY                                                                72567
NOT APPLICABLE                                                                       38452
IMPROPER OVERTAKING/PASSING                                                          35747
FAILING TO REDUCE SPEED TO AVOID CRASH                                               31048
IMPROPER BACKING                                                                     29473
IMPROPER LANE USAGE                                                                  26774
IMPROPER TURNING/NO SIGNAL                                                           24259
DRIVING SKILLS/KNOWLEDGE/EXPERIENCE                                                  24041
DISREGARDING TRAFFIC SIGNALS                                                         14223

In [14]:
df1['LOCATION'].value_counts()

POINT (-87.905309125103 41.976201139024)    1161
POINT (-87.619928173678 41.900958919109)     674
POINT (-87.580147768689 41.791420282098)     527
POINT (-87.585971992965 41.751460603167)     512
POINT (-87.585275565077 41.722257273006)     400
                                            ... 
POINT (-87.583105700753 41.746344711981)       1
POINT (-87.687225128171 41.903121223164)       1
POINT (-87.629569703337 41.706978349648)       1
POINT (-87.683712086838 41.970398127951)       1
POINT (-87.708443869236 41.777165573865)       1
Name: LOCATION, Length: 279315, dtype: int64

### lets further hone in on the relevant columns

In [29]:
df2 = df1[[
    'PERSON_TYPE',
    'SEX',
    'EJECTION',
    'INJURY_CLASSIFICATION',
    'DRIVER_VISION',
    'PHYSICAL_CONDITION',
    'POSTED_SPEED_LIMIT',
    'TRAFFIC_CONTROL_DEVICE',
    'DEVICE_CONDITION',
    'WEATHER_CONDITION',
    'LIGHTING_CONDITION',
    'FIRST_CRASH_TYPE',
    'TRAFFICWAY_TYPE',
    'ALIGNMENT',
    'ROADWAY_SURFACE_COND',
    'ROAD_DEFECT',
    'REPORT_TYPE',
    'CRASH_TYPE',
    'DAMAGE',
    'DATE_POLICE_NOTIFIED',
    'PRIM_CONTRIBUTORY_CAUSE',
    'STREET_NO',
    'STREET_DIRECTION',
    'STREET_NAME',
    'BEAT_OF_OCCURRENCE',
    'NUM_UNITS',
    'MOST_SEVERE_INJURY',
    'INJURIES_TOTAL',
    'INJURIES_FATAL',
    'INJURIES_INCAPACITATING',
    'INJURIES_NON_INCAPACITATING',
    'INJURIES_REPORTED_NOT_EVIDENT',
    'INJURIES_NO_INDICATION',
    'INJURIES_UNKNOWN',
    'CRASH_HOUR',
    'CRASH_DAY_OF_WEEK',
    'CRASH_MONTH',
    'LATITUDE',
    'LONGITUDE',
    'LOCATION',
    'CRASH_DATE',
    'UNIT_TYPE',
    'MAKE',
    'MODEL',
    'LIC_PLATE_STATE',
    'VEHICLE_YEAR',
    'VEHICLE_DEFECT',
    'VEHICLE_TYPE',
    'VEHICLE_USE',
    'TRAVEL_DIRECTION',
    'MANEUVER',
    'OCCUPANT_CNT',
    'FIRST_CONTACT_POINT',
    'AGE',
    'BAC_RESULT',
    'SAFETY_EQUIPMENT',
    'AIRBAG_DEPLOYED'
]]


In [16]:
df2.sample(20)

,AGE,MOST_SEVERE_INJURY,MAKE,MODEL,VEHICLE_YEAR,CRASH_DATE,CRASH_DAY_OF_WEEK,DAMAGE,VEHICLE_TYPE,BAC_RESULT,SAFETY_EQUIPMENT,LIGHTING_CONDITION,WEATHER_CONDITION,AIRBAG_DEPLOYED,ROAD_DEFECT,PRIM_CONTRIBUTORY_CAUSE,SEC_CONTRIBUTORY_CAUSE,FIRST_CONTACT_POINT,POSTED_SPEED_LIMIT,CRASH_HOUR
63567,38.0,NO INDICATION OF INJURY,HONDA,ACCORD,2014.0,11/11/2018 05:14:00 PM,1,"OVER $1,500",PASSENGER,NaN,SAFETY BELT USED,"DARKNESS, LIGHTED ROAD",CLEAR,"DEPLOYED, FRONT",NO DEFECTS,UNABLE TO DETERMINE,UNABLE TO DETERMINE,FRONT-RIGHT,35,17
268532,35.0,NO INDICATION OF INJURY,TOYOTA,CAMRY,1999.0,12/20/2021 03:00:00 PM,2,"$501 - $1,500",PASSENGER,TEST NOT OFFERED,SAFETY BELT USED,DAYLIGHT,CLEAR,NOT APPLICABLE,NO DEFECTS,DISREGARDING STOP SIGN,DISREGARDING STOP SIGN,FRONT-LEFT-CORNER,25,15
232707,NaN,NO INDICATION OF INJURY,AUDI,OTHER (EXPLAIN IN NARRATIVE),2007.0,11/13/2021 07:30:00 PM,7,"OVER $1,500",PASSENGER,TEST NOT OFFERED,USAGE UNKNOWN,DARKNESS,CLEAR,DEPLOYMENT UNKNOWN,NO DEFECTS,UNABLE TO DETERMINE,UNABLE TO DETERMINE,UNKNOWN,30,19
641019,NaN,NO INDICATION OF INJURY,UNKNOWN,OTHER (EXPLAIN IN NARRATIVE),NaN,04/19/2023 05:00:00 AM,4,"OVER $1,500",UNKNOWN/NA,TEST NOT OFFERED,USAGE UNKNOWN,"DARKNESS, LIGHTED ROAD",UNKNOWN,DEPLOYMENT UNKNOWN,NO DEFECTS,UNABLE TO DETERMINE,NOT APPLICABLE,UNKNOWN,25,5
359067,NaN,NO INDICATION OF INJURY,DATSUN,UNKNOWN,2017.0,10/05/2017 08:13:00 PM,5,$500 OR LESS,PASSENGER,TEST NOT OFFERED,USAGE UNKNOWN,"DARKNESS, LIGHTED ROAD",CLEAR,DID NOT DEPLOY,NO DEFECTS,UNABLE TO DETERMINE,UNABLE TO DETERMINE,FRONT-RIGHT,30,20
224085,NaN,NO INDICATION OF INJURY,UNKNOWN,UNKNOWN,NaN,12/18/2019 04:30:00 PM,4,"OVER $1,500",UNKNOWN/NA,NaN,USAGE UNKNOWN,DAYLIGHT,CLEAR,DID NOT DEPLOY,UNKNOWN,FAILING TO YIELD RIGHT-OF-WAY,FAILING TO REDUCE SPEED TO AVOID CRASH,UNKNOWN,30,16
24688,76.0,NO INDICATION OF INJURY,CHEVROLET,COBALT,2007.0,03/29/2020 12:45:00 PM,1,"OVER $1,500",PASSENGER,TEST NOT OFFERED,USAGE UNKNOWN,DAYLIGHT,CLEAR,DID NOT DEPLOY,NO DEFECTS,FOLLOWING TOO CLOSELY,UNABLE TO DETERMINE,FRONT-LEFT-CORNER,20,12
468975,NaN,NO INDICATION OF INJURY,LAND ROVER,LAND ROVER,NaN,08/24/2018 03:34:00 PM,6,"OVER $1,500",SPORT UTILITY VEHICLE (SUV),TEST NOT OFFERED,USAGE UNKNOWN,DAYLIGHT,CLEAR,"DEPLOYED, SIDE",NO DEFECTS,FAILING TO YIELD RIGHT-OF-WAY,NOT APPLICABLE,FRONT,30,15
378908,66.0,NO INDICATION OF INJURY,FORD,FORD,2004.0,12/08/2017 07:19:00 PM,6,"OVER $1,500",PASSENGER,TEST NOT OFFERED,SAFETY BELT NOT USED,"DARKNESS, LIGHTED ROAD",CLEAR,DID NOT DEPLOY,NO DEFECTS,UNABLE TO DETERMINE,UNABLE TO DETERMINE,NONE,30,19
421702,24.0,NO INDICATION OF INJURY,CHRYSLER,Sebring,2002.0,04/13/2018 03:15:00 PM,6,"OVER $1,500",PASSENGER,TEST NOT OFFERED,SAFETY BELT USED,DAYLIGHT,CLEAR,DID NOT DEPLOY,UNKNOWN,IMPROPER OVERTAKING/PASSING,UNABLE TO DETERMINE,SIDE-RIGHT,30,15


## lets do some quick cleaning up of outliers

In [30]:
df2['VEHICLE_YEAR'].value_counts()

2015.0    36281
2016.0    33811
2014.0    32851
2017.0    32795
2013.0    30706
          ...  
3016.0        1
3023.0        1
3027.0        1
5004.0        1
2054.0        1
Name: VEHICLE_YEAR, Length: 169, dtype: int64

In [31]:
#quickly cleaning up the outliers
df2 = df2[(df['VEHICLE_YEAR'] <= 2024) & (df['VEHICLE_YEAR'] >= 1930)]

In [32]:
df2['VEHICLE_YEAR'].value_counts().tail(20)
#looks clean

1977.0    16
1971.0    14
1974.0    11
1969.0    11
1973.0     9
1965.0     6
1964.0     6
1963.0     5
1966.0     5
1968.0     5
1967.0     3
1953.0     2
1952.0     2
1962.0     1
1961.0     1
1960.0     1
1958.0     1
1957.0     1
1954.0     1
1941.0     1
Name: VEHICLE_YEAR, dtype: int64

In [33]:
df2['AGE'].value_counts()

 25.0     12234
 26.0     11990
 27.0     11809
 24.0     11796
 28.0     11699
          ...  
 109.0        2
 102.0        2
-1.0          1
 110.0        1
 108.0        1
Name: AGE, Length: 110, dtype: int64

In [34]:
df2 = df2[(df['AGE'] <= 110) & (df['AGE'] >= 0)]

<ipython-input-34-61f8790cf53c>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df2 = df2[(df['AGE'] <= 110) & (df['AGE'] >= 0)]


In [35]:
df2['AGE'].value_counts().tail(20)

89.0     180
90.0     152
91.0      98
92.0      78
93.0      65
94.0      38
95.0      21
96.0      19
97.0      11
99.0      10
101.0      7
98.0       6
103.0      5
104.0      5
100.0      5
107.0      3
102.0      2
109.0      2
108.0      1
110.0      1
Name: AGE, dtype: int64

In [36]:
df2['CRASH_DAY_OF_WEEK'].value_counts()

6    65469
5    58372
4    57633
3    57587
7    56390
2    54284
1    45259
Name: CRASH_DAY_OF_WEEK, dtype: int64

In [37]:
df2.columns

Index(['PERSON_TYPE', 'SEX', 'EJECTION', 'INJURY_CLASSIFICATION',
       'DRIVER_VISION', 'PHYSICAL_CONDITION', 'POSTED_SPEED_LIMIT',
       'TRAFFIC_CONTROL_DEVICE', 'DEVICE_CONDITION', 'WEATHER_CONDITION',
       'LIGHTING_CONDITION', 'FIRST_CRASH_TYPE', 'TRAFFICWAY_TYPE',
       'ALIGNMENT', 'ROADWAY_SURFACE_COND', 'ROAD_DEFECT', 'REPORT_TYPE',
       'CRASH_TYPE', 'DAMAGE', 'DATE_POLICE_NOTIFIED',
       'PRIM_CONTRIBUTORY_CAUSE', 'STREET_NO', 'STREET_DIRECTION',
       'STREET_NAME', 'BEAT_OF_OCCURRENCE', 'NUM_UNITS', 'MOST_SEVERE_INJURY',
       'INJURIES_TOTAL', 'INJURIES_FATAL', 'INJURIES_INCAPACITATING',
       'INJURIES_NON_INCAPACITATING', 'INJURIES_REPORTED_NOT_EVIDENT',
       'INJURIES_NO_INDICATION', 'INJURIES_UNKNOWN', 'CRASH_HOUR',
       'CRASH_DAY_OF_WEEK', 'CRASH_MONTH', 'LATITUDE', 'LONGITUDE', 'LOCATION',
       'CRASH_DATE', 'UNIT_TYPE', 'MAKE', 'MODEL', 'LIC_PLATE_STATE',
       'VEHICLE_YEAR', 'VEHICLE_DEFECT', 'VEHICLE_TYPE', 'VEHICLE_USE',
       'TRAVEL_DIRE

In [38]:
pd.set_option('display.max_rows', 1307)
df2.groupby('MOST_SEVERE_INJURY')['VEHICLE_YEAR'].value_counts().reset_index(name='count')

,MOST_SEVERE_INJURY,VEHICLE_YEAR,count
0,FATAL,2015.0,31
1,FATAL,2003.0,29
2,FATAL,2008.0,29
3,FATAL,2005.0,28
4,FATAL,2016.0,27
5,FATAL,2017.0,25
6,FATAL,2007.0,24
7,FATAL,2012.0,24
8,FATAL,2018.0,23
9,FATAL,2006.0,22


In [39]:
df2['MAKE'].value_counts()

CHEVROLET                                                       49842
FORD                                                            47011
NISSAN                                                          33041
HONDA                                                           30642
TOYOTA MOTOR COMPANY, LTD.                                      26856
TOYOTA                                                          19917
DODGE                                                           19453
JEEP                                                            18157
HYUNDAI                                                         17206
CHRYSLER                                                         8347
VOLKSWAGEN                                                       8197
BUICK                                                            7321
LEXUS                                                            6122
MAZDA                                                            5516
MERCEDES-BENZ       

In [40]:
# new columns
df2['FATAL_INCAPACITATING'] = (df2['MOST_SEVERE_INJURY'] == 'FATAL') | (df2['MOST_SEVERE_INJURY'] == 'INCAPACITATING INJURY')
df2['NO_INJURY_NONINCAPACITATING'] = (df2['MOST_SEVERE_INJURY'] == 'NO INDICATION OF INJURY') | (df2['MOST_SEVERE_INJURY'] == 'NONINCAPACITATING INJURY')

In [181]:
dfx = df[df['MOST_SEVERE_INJURY'].isin(['FATAL', 'INCAPACITATING INJURY', 'NO INDICATION OF INJURY', 'NONINCAPACITATING INJURY'])]

# 2. Group by 'MAKE' and 'MOST_SEVERE_INJURY', and count the number of rows in each group
dfx = dfx.groupby(['MAKE', 'MOST_SEVERE_INJURY']).size().unstack(fill_value=0)

# 2.5 Filter the grouped DataFrame to only include 'MAKE' values with a total count of 50 or more
dfx = dfx[dfx.sum(axis=1) >= 5000]

# 3. Calculate the ratios for each 'MAKE'
dfx['RATIO'] = (dfx['FATAL'] + dfx['INCAPACITATING INJURY']) / (dfx['NO INDICATION OF INJURY'] + dfx['NONINCAPACITATING INJURY'])

dfx.sort_values(by='RATIO', ascending=False)


MOST_SEVERE_INJURY,FATAL,INCAPACITATING INJURY,NO INDICATION OF INJURY,NONINCAPACITATING INJURY,RATIO
MAKE,,,,,
PONTIAC,15,160,6927,711,0.022912
BUICK,13,233,9935,976,0.022546
KIA MOTORS CORP,6,136,6276,582,0.020706
HYUNDAI,23,464,21694,2101,0.020466
CADILLAC,16,116,5969,588,0.020131
DODGE,39,538,26788,2390,0.019775
INFINITI,15,108,5638,593,0.019740
KIA,7,142,6907,736,0.019495
GENERAL MOTORS CORPORATION (GMC),6,97,4982,405,0.019120


In [90]:
df2

,AGE,MOST_SEVERE_INJURY,MAKE,MODEL,VEHICLE_YEAR,CRASH_DATE,CRASH_DAY_OF_WEEK,DAMAGE,VEHICLE_TYPE,BAC_RESULT,SAFETY_EQUIPMENT,VEHICLE_TYPE,LIGHTING_CONDITION,AIRBAG_DEPLOYED,ROAD_DEFECT,PRIM_CONTRIBUTORY_CAUSE,SEC_CONTRIBUTORY_CAUSE,FIRST_CONTACT_POINT,CRASH_HOUR
0,25.0,NO INDICATION OF INJURY,HONDA,CIVIC,2016.0,09/28/2019 03:30:00 AM,7,"OVER $1,500",PASSENGER,TEST NOT OFFERED,NONE PRESENT,PASSENGER,DARKNESS,DEPLOYMENT UNKNOWN,NO DEFECTS,UNABLE TO DETERMINE,NOT APPLICABLE,FRONT,3
12,34.0,NO INDICATION OF INJURY,"TOYOTA MOTOR COMPANY, LTD.",CAMRY,2001.0,02/23/2020 06:15:00 PM,1,"OVER $1,500",PASSENGER,TEST NOT OFFERED,SAFETY BELT USED,PASSENGER,DUSK,DID NOT DEPLOY,NO DEFECTS,IMPROPER OVERTAKING/PASSING,UNABLE TO DETERMINE,FRONT-LEFT-CORNER,18
13,53.0,NO INDICATION OF INJURY,NEW FLYER,NEW FLYER,2007.0,04/30/2020 03:05:00 PM,5,"$501 - $1,500",BUS OVER 15 PASS.,TEST NOT OFFERED,SAFETY BELT USED,BUS OVER 15 PASS.,DAYLIGHT,DID NOT DEPLOY,NO DEFECTS,UNABLE TO DETERMINE,NOT APPLICABLE,SIDE-LEFT,15
17,33.0,NO INDICATION OF INJURY,HONDA,ACCORD,2003.0,05/05/2020 11:35:00 AM,3,"OVER $1,500",PASSENGER,TEST NOT OFFERED,USAGE UNKNOWN,PASSENGER,DAYLIGHT,"DEPLOYED, COMBINATION",NO DEFECTS,UNABLE TO DETERMINE,NOT APPLICABLE,FRONT-RIGHT-CORNER,11
18,22.0,NO INDICATION OF INJURY,TOYOTA,AVALON,2006.0,05/06/2020 12:53:00 AM,4,"$501 - $1,500",PASSENGER,TEST NOT OFFERED,SAFETY BELT USED,PASSENGER,"DARKNESS, LIGHTED ROAD",DID NOT DEPLOY,NO DEFECTS,IMPROPER LANE USAGE,UNABLE TO DETERMINE,FRONT-RIGHT-CORNER,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
731899,44.0,NO INDICATION OF INJURY,HONDA,ODYSSEY,2014.0,04/18/2023 05:00:00 PM,3,"$501 - $1,500",SPORT UTILITY VEHICLE (SUV),TEST NOT OFFERED,SAFETY BELT USED,SPORT UTILITY VEHICLE (SUV),DAYLIGHT,DID NOT DEPLOY,NO DEFECTS,IMPROPER LANE USAGE,NOT APPLICABLE,SIDE-LEFT,17
731900,54.0,NO INDICATION OF INJURY,TOYOTA,4RUNNER,2015.0,04/18/2023 12:20:00 PM,3,"$501 - $1,500",PASSENGER,TEST NOT OFFERED,SAFETY BELT USED,PASSENGER,DAYLIGHT,DID NOT DEPLOY,OTHER,UNABLE TO DETERMINE,UNABLE TO DETERMINE,OTHER,12
731902,23.0,"REPORTED, NOT EVIDENT",JEEP,PATRIOT,2015.0,04/18/2023 11:32:00 AM,3,"OVER $1,500",PASSENGER,TEST NOT OFFERED,USAGE UNKNOWN,PASSENGER,DAYLIGHT,"DEPLOYED, COMBINATION",NO DEFECTS,TURNING RIGHT ON RED,NOT APPLICABLE,SIDE-RIGHT,11
731904,41.0,NO INDICATION OF INJURY,FORD,OTHER (EXPLAIN IN NARRATIVE),2005.0,04/18/2023 07:12:00 PM,3,"$501 - $1,500",PICKUP,TEST NOT OFFERED,SAFETY BELT USED,PICKUP,DAYLIGHT,NOT APPLICABLE,NO DEFECTS,FAILING TO YIELD RIGHT-OF-WAY,NOT APPLICABLE,FRONT-RIGHT-CORNER,19


In [97]:
df2.sample(300)

,AGE,MOST_SEVERE_INJURY,MAKE,MODEL,VEHICLE_YEAR,CRASH_DATE,CRASH_DAY_OF_WEEK,DAMAGE,VEHICLE_TYPE,BAC_RESULT,...,VEHICLE_TYPE,LIGHTING_CONDITION,AIRBAG_DEPLOYED,ROAD_DEFECT,PRIM_CONTRIBUTORY_CAUSE,SEC_CONTRIBUTORY_CAUSE,FIRST_CONTACT_POINT,CRASH_HOUR,FATAL_INCAPACITATING,NO_INJURY_NONINCAPACITATING
404334,42.0,NO INDICATION OF INJURY,FORD,FUSION,2016.0,02/20/2018 03:25:00 PM,3,$500 OR LESS,PASSENGER,TEST NOT OFFERED,...,PASSENGER,DAYLIGHT,DID NOT DEPLOY,NO DEFECTS,IMPROPER LANE USAGE,UNABLE TO DETERMINE,FRONT-RIGHT,15,False,True
192203,32.0,NO INDICATION OF INJURY,JEEP,LIBERTY,2011.0,06/08/2021 03:40:00 PM,3,"OVER $1,500",PASSENGER,TEST NOT OFFERED,...,PASSENGER,DAYLIGHT,DID NOT DEPLOY,NO DEFECTS,FAILING TO YIELD RIGHT-OF-WAY,NOT APPLICABLE,SIDE-RIGHT-FRONT,15,False,True
419945,49.0,NO INDICATION OF INJURY,ACURA (DIV. OF AMERICAN HONDA MOTOR CO.),MDX,2014.0,04/08/2018 01:35:00 PM,1,$500 OR LESS,PASSENGER,TEST NOT OFFERED,...,PASSENGER,DAYLIGHT,DEPLOYMENT UNKNOWN,UNKNOWN,UNABLE TO DETERMINE,NOT APPLICABLE,SIDE-RIGHT,13,False,True
375696,27.0,NO INDICATION OF INJURY,HYUNDAI,SONATA,2017.0,11/30/2017 07:35:00 AM,5,"$501 - $1,500",PASSENGER,TEST NOT OFFERED,...,PASSENGER,DAYLIGHT,DID NOT DEPLOY,NO DEFECTS,IMPROPER OVERTAKING/PASSING,NOT APPLICABLE,REAR-LEFT,7,False,True
30152,41.0,NO INDICATION OF INJURY,DODGE,OTHER (EXPLAIN IN NARRATIVE),2001.0,04/01/2020 06:41:00 PM,4,"OVER $1,500",PICKUP,TEST NOT OFFERED,...,PICKUP,DAYLIGHT,DID NOT DEPLOY,NO DEFECTS,IMPROPER OVERTAKING/PASSING,UNABLE TO DETERMINE,SIDE-RIGHT,18,False,True
479447,21.0,NO INDICATION OF INJURY,CHEVROLET,COBALT,2013.0,10/01/2018 11:15:00 AM,2,"OVER $1,500",PASSENGER,TEST NOT OFFERED,...,PASSENGER,DAYLIGHT,"DEPLOYED, FRONT",NO DEFECTS,FOLLOWING TOO CLOSELY,NOT APPLICABLE,FRONT,11,False,True
249069,22.0,NONINCAPACITATING INJURY,PONTIAC,GRAND PRIX,2007.0,11/24/2021 05:42:00 PM,4,"OVER $1,500",PASSENGER,TEST NOT OFFERED,...,PASSENGER,"DARKNESS, LIGHTED ROAD",NOT APPLICABLE,NO DEFECTS,UNABLE TO DETERMINE,UNABLE TO DETERMINE,SIDE-LEFT,17,False,True
404759,62.0,NO INDICATION OF INJURY,CADILLAC,SRX,2014.0,02/21/2018 06:30:00 PM,4,"OVER $1,500",SPORT UTILITY VEHICLE (SUV),TEST NOT OFFERED,...,SPORT UTILITY VEHICLE (SUV),UNKNOWN,DEPLOYMENT UNKNOWN,UNKNOWN,UNABLE TO DETERMINE,NOT APPLICABLE,FRONT-LEFT,18,False,True
350760,59.0,NO INDICATION OF INJURY,"TOYOTA MOTOR COMPANY, LTD.",CAMRY,2014.0,09/20/2017 08:21:00 AM,4,"$501 - $1,500",PASSENGER,TEST NOT OFFERED,...,PASSENGER,DAYLIGHT,DID NOT DEPLOY,NO DEFECTS,IMPROPER OVERTAKING/PASSING,UNABLE TO DETERMINE,SIDE-RIGHT,8,False,True
587832,45.0,NO INDICATION OF INJURY,FORD,F150,2018.0,11/21/2019 06:30:00 AM,5,"$501 - $1,500",PICKUP,TEST NOT OFFERED,...,PICKUP,DAYLIGHT,DID NOT DEPLOY,NO DEFECTS,IMPROPER OVERTAKING/PASSING,UNABLE TO DETERMINE,FRONT,6,False,True


In [98]:
df2.columns

Index(['AGE', 'MOST_SEVERE_INJURY', 'MAKE', 'MODEL', 'VEHICLE_YEAR',
       'CRASH_DATE', 'CRASH_DAY_OF_WEEK', 'DAMAGE', 'VEHICLE_TYPE',
       'BAC_RESULT', 'SAFETY_EQUIPMENT', 'VEHICLE_TYPE', 'LIGHTING_CONDITION',
       'AIRBAG_DEPLOYED', 'ROAD_DEFECT', 'PRIM_CONTRIBUTORY_CAUSE',
       'SEC_CONTRIBUTORY_CAUSE', 'FIRST_CONTACT_POINT', 'CRASH_HOUR',
       'FATAL_INCAPACITATING', 'NO_INJURY_NONINCAPACITATING'],
      dtype='object')

In [99]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 394994 entries, 0 to 731905
Data columns (total 21 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   AGE                          394994 non-null  float64
 1   MOST_SEVERE_INJURY           394994 non-null  object 
 2   MAKE                         394994 non-null  object 
 3   MODEL                        394954 non-null  object 
 4   VEHICLE_YEAR                 394994 non-null  float64
 5   CRASH_DATE                   394994 non-null  object 
 6   CRASH_DAY_OF_WEEK            394994 non-null  int64  
 7   DAMAGE                       394994 non-null  object 
 8   VEHICLE_TYPE                 394994 non-null  object 
 9   BAC_RESULT                   362788 non-null  object 
 10  SAFETY_EQUIPMENT             394978 non-null  object 
 11  VEHICLE_TYPE                 394994 non-null  object 
 12  LIGHTING_CONDITION           394994 non-null  object 
 13 

In [370]:
df2['BAC_RESULT'].value_counts()

TEST NOT OFFERED                   354396
TEST REFUSED                         5232
TEST TAKEN                           1686
TEST PERFORMED, RESULTS UNKNOWN      1474
Name: BAC_RESULT, dtype: int64

In [371]:
df2.shape

(394994, 20)

In [372]:
len(df2.loc[(df2['PRIM_CONTRIBUTORY_CAUSE'] =='UNABLE TO DETERMINE') | (df2['PRIM_CONTRIBUTORY_CAUSE'] == 'NOT APPLICABLE')])/df2.shape[0]

0.34794958910768264

In [373]:
df2['PRIM_CONTRIBUTORY_CAUSE'].value_counts()

UNABLE TO DETERMINE                                                                 120310
FAILING TO YIELD RIGHT-OF-WAY                                                        54067
FOLLOWING TOO CLOSELY                                                                44921
FAILING TO REDUCE SPEED TO AVOID CRASH                                               20164
IMPROPER OVERTAKING/PASSING                                                          17886
NOT APPLICABLE                                                                       17128
IMPROPER BACKING                                                                     16356
IMPROPER TURNING/NO SIGNAL                                                           15945
IMPROPER LANE USAGE                                                                  15238
DRIVING SKILLS/KNOWLEDGE/EXPERIENCE                                                  14802
DISREGARDING TRAFFIC SIGNALS                                                          8834

In [374]:
target_dict = {
    'reckless driving': [
        'FOLLOWING TOO CLOSELY', 
        'IMPROPER OVERTAKING/PASSING', 
        'EXCEEDING SAFE SPEED FOR CONDITIONS',
        'EXCEEDING AUTHORIZED SPEED LIMIT', 
        'IMPROPER TURNING/NO SIGNAL', 
        'IMPROPER BACKING',  
        'IMPROPER LANE USAGE', 
        'FAILING TO REDUCE SPEED TO AVOID CRASH', 
        'OPERATING VEHICLE IN ERRATIC, RECKLESS, CARELESS, NEGLIGENT OR AGGRESSIVE MANNER',
        'TURNING RIGHT ON RED', 
        'DRIVING ON WRONG SIDE/WRONG WAY'
        'CELL PHONE USE OTHER THAN TEXTING', 
        'TEXTING',
        'DISTRACTION - OTHER ELECTRONIC DEVICE (NAVIGATION DEVICE, DVD PLAYER, ETC.)',
        'DISTRACTION - FROM OUTSIDE VEHICLE', 
        'DISTRACTION - FROM INSIDE VEHICLE'
    ],

    'missed or misunderstood traffic signs': [
        'DISREGARDING YIELD SIGN', 
        'FAILING TO YIELD RIGHT-OF-WAY', 
        'DISREGARDING TRAFFIC SIGNALS', 
        'DISREGARDING OTHER TRAFFIC SIGNS',
        'DISREGARDING ROAD MARKINGS', 
        'DISREGARDING STOP SIGN'
    ],

    'issue with the driver or vehicle': [
        'DRIVING SKILLS/KNOWLEDGE/EXPERIENCE', 
        'PHYSICAL CONDITION OF DRIVER', 
        'EQUIPMENT - VEHICLE CONDITION', 
        'HAD BEEN DRINKING (USE WHEN ARREST IS NOT MADE)',
        'UNDER THE INFLUENCE OF ALCOHOL/DRUGS (USE WHEN ARREST IS EFFECTED)'
    ],

    'external variable': [
        'WEATHER', 
        'VISION OBSCURED (SIGNS, TREE LIMBS, BUILDINGS, ETC.)', 
        'EVASIVE ACTION DUE TO ANIMAL, OBJECT, NONMOTORIST',
        'ROAD CONSTRUCTION/MAINTENANCE', 
        'ROAD ENGINEERING/SURFACE/MARKING DEFECTS', 
        'ANIMAL', 
        'OBSTRUCTED CROSSWALKS',
    ],

    'drop': [
        'PASSING STOPPED SCHOOL BUS', 
        'MOTORCYCLE ADVANCING LEGALLY ON RED LIGHT',
        'BICYCLE ADVANCING LEGALLY ON RED LIGHT', 
        'RELATED TO BUS STOP'
    ]
}


In [375]:
target_dict

{'reckless driving': ['FOLLOWING TOO CLOSELY',
  'IMPROPER OVERTAKING/PASSING',
  'EXCEEDING SAFE SPEED FOR CONDITIONS',
  'EXCEEDING AUTHORIZED SPEED LIMIT',
  'IMPROPER TURNING/NO SIGNAL',
  'IMPROPER BACKING',
  'IMPROPER LANE USAGE',
  'FAILING TO REDUCE SPEED TO AVOID CRASH',
  'OPERATING VEHICLE IN ERRATIC, RECKLESS, CARELESS, NEGLIGENT OR AGGRESSIVE MANNER',
  'TURNING RIGHT ON RED',
  'DRIVING ON WRONG SIDE/WRONG WAYCELL PHONE USE OTHER THAN TEXTING',
  'TEXTING',
  'DISTRACTION - OTHER ELECTRONIC DEVICE (NAVIGATION DEVICE, DVD PLAYER, ETC.)',
  'DISTRACTION - FROM OUTSIDE VEHICLE',
  'DISTRACTION - FROM INSIDE VEHICLE'],
 'missed or misunderstood traffic signs': ['DISREGARDING YIELD SIGN',
  'FAILING TO YIELD RIGHT-OF-WAY',
  'DISREGARDING TRAFFIC SIGNALS',
  'DISREGARDING OTHER TRAFFIC SIGNS',
  'DISREGARDING ROAD MARKINGS',
  'DISREGARDING STOP SIGN'],
 'issue with the driver or vehicle': ['DRIVING SKILLS/KNOWLEDGE/EXPERIENCE',
  'PHYSICAL CONDITION OF DRIVER',
  'EQUIPMENT 

In [376]:
new_dict = {value: key for key in target_dict for value in target_dict[key]}

In [377]:
df2['target_mapped'] = df2['PRIM_CONTRIBUTORY_CAUSE']

In [378]:
df2['target_mapped'] = df2['target_mapped'].map(new_dict)

In [379]:
df2['target_mapped'].value_counts()

reckless driving                         143420
missed or misunderstood traffic signs     68788
issue with the driver or vehicle          26041
external variable                         16586
drop                                        249
Name: target_mapped, dtype: int64

In [380]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 394994 entries, 0 to 731905
Data columns (total 21 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   AGE                      394994 non-null  float64
 1   MOST_SEVERE_INJURY       394994 non-null  object 
 2   MAKE                     394994 non-null  object 
 3   MODEL                    394954 non-null  object 
 4   VEHICLE_YEAR             394994 non-null  float64
 5   CRASH_DATE               394994 non-null  object 
 6   CRASH_DAY_OF_WEEK        394994 non-null  int64  
 7   DAMAGE                   394994 non-null  object 
 8   VEHICLE_TYPE             394994 non-null  object 
 9   BAC_RESULT               362788 non-null  object 
 10  SAFETY_EQUIPMENT         394978 non-null  object 
 11  LIGHTING_CONDITION       394994 non-null  object 
 12  WEATHER_CONDITION        394994 non-null  object 
 13  AIRBAG_DEPLOYED          394695 non-null  object 
 14  ROAD

In [381]:
df2 = df2[df2['target_mapped'] != 'drop']

In [382]:
df2 = df2.dropna()

In [383]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 231850 entries, 12 to 731904
Data columns (total 21 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   AGE                      231850 non-null  float64
 1   MOST_SEVERE_INJURY       231850 non-null  object 
 2   MAKE                     231850 non-null  object 
 3   MODEL                    231850 non-null  object 
 4   VEHICLE_YEAR             231850 non-null  float64
 5   CRASH_DATE               231850 non-null  object 
 6   CRASH_DAY_OF_WEEK        231850 non-null  int64  
 7   DAMAGE                   231850 non-null  object 
 8   VEHICLE_TYPE             231850 non-null  object 
 9   BAC_RESULT               231850 non-null  object 
 10  SAFETY_EQUIPMENT         231850 non-null  object 
 11  LIGHTING_CONDITION       231850 non-null  object 
 12  WEATHER_CONDITION        231850 non-null  object 
 13  AIRBAG_DEPLOYED          231850 non-null  object 
 14  ROA

In [330]:
df2.head()

,AGE,MOST_SEVERE_INJURY,MAKE,MODEL,VEHICLE_YEAR,CRASH_DATE,CRASH_DAY_OF_WEEK,DAMAGE,VEHICLE_TYPE,BAC_RESULT,...,LIGHTING_CONDITION,AIRBAG_DEPLOYED,ROAD_DEFECT,PRIM_CONTRIBUTORY_CAUSE,SEC_CONTRIBUTORY_CAUSE,FIRST_CONTACT_POINT,CRASH_HOUR,FATAL_INCAPACITATING,NO_INJURY_NONINCAPACITATING,target_mapped
12,34.0,NO INDICATION OF INJURY,TOYOTA,CAMRY,2001.0,02/23/2020 06:15:00 PM,1,"OVER $1,500",PASSENGER,TEST NOT OFFERED,...,DUSK,DID NOT DEPLOY,NO DEFECTS,IMPROPER OVERTAKING/PASSING,UNABLE TO DETERMINE,FRONT-LEFT-CORNER,18,False,True,reckless driving
18,22.0,NO INDICATION OF INJURY,TOYOTA,AVALON,2006.0,05/06/2020 12:53:00 AM,4,"$501 - $1,500",PASSENGER,TEST NOT OFFERED,...,"DARKNESS, LIGHTED ROAD",DID NOT DEPLOY,NO DEFECTS,IMPROPER LANE USAGE,UNABLE TO DETERMINE,FRONT-RIGHT-CORNER,0,False,True,reckless driving
37,22.0,NO INDICATION OF INJURY,CHEVROLET,UNKNOWN,2013.0,02/23/2020 06:22:00 PM,1,"OVER $1,500",PASSENGER,TEST NOT OFFERED,...,DARKNESS,"DEPLOYED, FRONT",UNKNOWN,IMPROPER LANE USAGE,NOT APPLICABLE,SIDE-RIGHT-FRONT,18,False,True,reckless driving
38,61.0,NO INDICATION OF INJURY,CHEVROLET,SILVERADO,2017.0,05/02/2020 11:00:00 AM,7,"OVER $1,500",SPORT UTILITY VEHICLE (SUV),TEST NOT OFFERED,...,DAYLIGHT,DID NOT DEPLOY,NO DEFECTS,FOLLOWING TOO CLOSELY,NOT APPLICABLE,REAR,11,False,True,reckless driving
42,28.0,NO INDICATION OF INJURY,HONDA,CR-V,2002.0,03/25/2022 05:10:00 AM,6,"OVER $1,500",PASSENGER,TEST NOT OFFERED,...,"DARKNESS, LIGHTED ROAD","DEPLOYED, COMBINATION",NO DEFECTS,FAILING TO YIELD RIGHT-OF-WAY,NOT APPLICABLE,FRONT-LEFT-CORNER,5,False,True,missed or misunderstood traffic signs


In [384]:
df2['DAMAGE'].value_counts()

OVER $1,500      154319
$501 - $1,500     55256
$500 OR LESS      22275
Name: DAMAGE, dtype: int64

In [332]:
df2.groupby('MOST_SEVERE_INJURY').count()

,AGE,MAKE,MODEL,VEHICLE_YEAR,CRASH_DATE,CRASH_DAY_OF_WEEK,DAMAGE,VEHICLE_TYPE,BAC_RESULT,SAFETY_EQUIPMENT,LIGHTING_CONDITION,AIRBAG_DEPLOYED,ROAD_DEFECT,PRIM_CONTRIBUTORY_CAUSE,SEC_CONTRIBUTORY_CAUSE,FIRST_CONTACT_POINT,CRASH_HOUR,FATAL_INCAPACITATING,NO_INJURY_NONINCAPACITATING,target_mapped
MOST_SEVERE_INJURY,,,,,,,,,,,,,,,,,,,,
FATAL,244,244,244,244,244,244,244,244,244,244,244,244,244,244,244,244,244,244,244,244
INCAPACITATING INJURY,4503,4503,4503,4503,4503,4503,4503,4503,4503,4503,4503,4503,4503,4503,4503,4503,4503,4503,4503,4503
NO INDICATION OF INJURY,192103,192103,192103,192103,192103,192103,192103,192103,192103,192103,192103,192103,192103,192103,192103,192103,192103,192103,192103,192103
NONINCAPACITATING INJURY,21211,21211,21211,21211,21211,21211,21211,21211,21211,21211,21211,21211,21211,21211,21211,21211,21211,21211,21211,21211
"REPORTED, NOT EVIDENT",11534,11534,11534,11534,11534,11534,11534,11534,11534,11534,11534,11534,11534,11534,11534,11534,11534,11534,11534,11534


In [333]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 229595 entries, 12 to 731904
Data columns (total 21 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   AGE                          229595 non-null  float64
 1   MOST_SEVERE_INJURY           229595 non-null  object 
 2   MAKE                         229595 non-null  object 
 3   MODEL                        229595 non-null  object 
 4   VEHICLE_YEAR                 229595 non-null  float64
 5   CRASH_DATE                   229595 non-null  object 
 6   CRASH_DAY_OF_WEEK            229595 non-null  int64  
 7   DAMAGE                       229595 non-null  object 
 8   VEHICLE_TYPE                 229595 non-null  object 
 9   BAC_RESULT                   229595 non-null  object 
 10  SAFETY_EQUIPMENT             229595 non-null  object 
 11  LIGHTING_CONDITION           229595 non-null  object 
 12  AIRBAG_DEPLOYED              229595 non-null  object 
 13

# cleaning up the make and vehicle type

In [385]:
make_counts = df2['MAKE'].value_counts()
over_50 = make_counts[make_counts > 50].index
df2 = df2[df2['MAKE'].isin(over_50)]

In [486]:
df2['MAKE'].value_counts()

CHEVROLET                                              29447
TOYOTA                                                 28051
FORD                                                   27059
NISSAN                                                 19577
HONDA                                                  18403
DODGE                                                  11327
JEEP                                                   10796
HYUNDAI                                                10222
KIA                                                     6111
VOLKSWAGEN                                              4901
CHRYSLER                                                4876
GENERAL MOTORS CORP.                                    4662
BUICK                                                   4303
LEXUS                                                   3633
MAZDA                                                   3249
MERCEDES-BENZ                                           3085
BMW                     

In [487]:
df2['MAKE'] = df2['MAKE'].replace({'TOYOTA MOTOR COMPANY, LTD.' : 'TOYOTA' , 
                 'KIA MOTORS CORP' : 'KIA',
                'GENERAL MOTORS CORPORATION (GMC)': "GENERAL MOTORS CORP.",
                'TESLA MOTORS': 'TESLA',
                'LINCOLN-CONTINENTAL':'LINCOLN',
                'INTERNATIONAL TRAILER CORP.': 'INTERNATIONAL TRAILER CORPORATION',
                'PETERBILT MOTORS CO., DIVISION PACCAR, INC.' : 'PETERBILT MOTORS COMPANY (DIVISION OF PACCAR, INC.)',
                'FREIGHTLINER CORP.' : 'FREIGHTLINER CORPORATION',
                'ACURA (DIV. OF AMERICAN HONDA MOTOR CO.)' : 'ACURA',
                'KENWORTH MOTOR TRUCK CO.' : 'KENWORTH MOTOR TRUCK COMPANY'
                
                                  })

In [488]:
df2['VEHICLE_TYPE'].value_counts()

PASSENGER                         156382
SPORT UTILITY VEHICLE (SUV)        33537
VAN/MINI-VAN                       12700
PICKUP                              8495
TRUCK - SINGLE UNIT                 5742
UNKNOWN/NA                          3562
TRACTOR W/ SEMI-TRAILER             2999
OTHER                               2233
BUS OVER 15 PASS.                   1837
BUS UP TO 15 PASS.                   558
MOTORCYCLE (OVER 150CC)              499
TRACTOR W/O SEMI-TRAILER             344
SINGLE UNIT TRUCK WITH TRAILER       333
OTHER VEHICLE WITH TRAILER           291
MOTOR DRIVEN CYCLE                    30
MOPED OR MOTORIZED BICYCLE            30
AUTOCYCLE                             23
Name: VEHICLE_TYPE, dtype: int64

In [489]:
VT_counts = df2['VEHICLE_TYPE'].value_counts()
over_20 = VT_counts[VT_counts > 20].index
df2 = df2[df2['VEHICLE_TYPE'].isin(over_20)]

In [490]:
df2['VEHICLE_TYPE'].value_counts()

PASSENGER                         156382
SPORT UTILITY VEHICLE (SUV)        33537
VAN/MINI-VAN                       12700
PICKUP                              8495
TRUCK - SINGLE UNIT                 5742
UNKNOWN/NA                          3562
TRACTOR W/ SEMI-TRAILER             2999
OTHER                               2233
BUS OVER 15 PASS.                   1837
BUS UP TO 15 PASS.                   558
MOTORCYCLE (OVER 150CC)              499
TRACTOR W/O SEMI-TRAILER             344
SINGLE UNIT TRUCK WITH TRAILER       333
OTHER VEHICLE WITH TRAILER           291
MOTOR DRIVEN CYCLE                    30
MOPED OR MOTORIZED BICYCLE            30
AUTOCYCLE                             23
Name: VEHICLE_TYPE, dtype: int64

## Now to baseline modeling

In [491]:
#numeric_pipeline = Pipeline([('numnorm', StandardScaler())])

#ordinal_pipeline = Pipeline([
  #  ('ordenc', OrdinalEncoder(categories = [columns])),
  #  ('ordnorm', StandardScaler())])

nominal_pipeline = Pipeline([
    ('onehotenc', OneHotEncoder(sparse = False, handle_unknown = 'ignore')), 
    ('onehotnorm', MaxAbsScaler())])


In [492]:
cat_cols = ['MAKE', 'MOST_SEVERE_INJURY', 'DAMAGE', 'VEHICLE_TYPE']

In [493]:
ct = ColumnTransformer(
    [ ("nominalpipe", nominal_pipeline, cat_cols)])

In [494]:
pipe = Pipeline([('preprocess', ct),
                      ('model',
                       LogisticRegression(max_iter = 100000))])
pipe

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('nominalpipe',
                                                  Pipeline(steps=[('onehotenc',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False)),
                                                                  ('onehotnorm',
                                                                   MaxAbsScaler())]),
                                                  ['MAKE', 'MOST_SEVERE_INJURY',
                                                   'DAMAGE',
                                                   'VEHICLE_TYPE'])])),
                ('model', LogisticRegression(max_iter=100000))])

In [495]:
df2['target_mapped'].value_counts()

reckless driving                         129278
missed or misunderstood traffic signs     61535
issue with the driver or vehicle          23647
external variable                         15135
Name: target_mapped, dtype: int64

In [496]:
df3 = df2[['MAKE', 'MOST_SEVERE_INJURY', 'DAMAGE', 'VEHICLE_TYPE', 'target_mapped']]

In [497]:
df3['target_mapped'].value_counts()

reckless driving                         129278
missed or misunderstood traffic signs     61535
issue with the driver or vehicle          23647
external variable                         15135
Name: target_mapped, dtype: int64

In [498]:
X = df3.drop('target_mapped', axis=1)
y = df3['target_mapped']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

pipe.fit(X_train, y_train)


Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('nominalpipe',
                                                  Pipeline(steps=[('onehotenc',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False)),
                                                                  ('onehotnorm',
                                                                   MaxAbsScaler())]),
                                                  ['MAKE', 'MOST_SEVERE_INJURY',
                                                   'DAMAGE',
                                                   'VEHICLE_TYPE'])])),
                ('model', LogisticRegression(max_iter=100000))])

In [499]:
print(pipe.score(X_train,y_train))
pipe.score(X_test,y_test)

0.5651963239617588


0.5641455606611642

In [500]:
preds = pipe.predict(X_test)

confusion_matrix(y_test, preds)

array([[    2,     0,   191,  2820],
       [    0,     1,   508,  4277],
       [    0,     4,  1514, 10759],
       [    2,     0,  1453, 24388]])

# More cleaning and feature engineering

In [501]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 229595 entries, 12 to 731904
Data columns (total 21 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   AGE                      229595 non-null  float64
 1   MOST_SEVERE_INJURY       229595 non-null  object 
 2   MAKE                     229595 non-null  object 
 3   MODEL                    229595 non-null  object 
 4   VEHICLE_YEAR             229595 non-null  float64
 5   CRASH_DATE               229595 non-null  object 
 6   CRASH_DAY_OF_WEEK        229595 non-null  int64  
 7   DAMAGE                   229595 non-null  object 
 8   VEHICLE_TYPE             229595 non-null  object 
 9   BAC_RESULT               229595 non-null  object 
 10  SAFETY_EQUIPMENT         229595 non-null  object 
 11  LIGHTING_CONDITION       229595 non-null  object 
 12  WEATHER_CONDITION        229595 non-null  object 
 13  AIRBAG_DEPLOYED          229595 non-null  object 
 14  ROA

In [502]:
df2.columns

Index(['AGE', 'MOST_SEVERE_INJURY', 'MAKE', 'MODEL', 'VEHICLE_YEAR',
       'CRASH_DATE', 'CRASH_DAY_OF_WEEK', 'DAMAGE', 'VEHICLE_TYPE',
       'BAC_RESULT', 'SAFETY_EQUIPMENT', 'LIGHTING_CONDITION',
       'WEATHER_CONDITION', 'AIRBAG_DEPLOYED', 'ROAD_DEFECT',
       'PRIM_CONTRIBUTORY_CAUSE', 'SEC_CONTRIBUTORY_CAUSE',
       'FIRST_CONTACT_POINT', 'POSTED_SPEED_LIMIT', 'CRASH_HOUR',
       'target_mapped'],
      dtype='object')

In [394]:
df2

,AGE,MOST_SEVERE_INJURY,MAKE,MODEL,VEHICLE_YEAR,CRASH_DATE,CRASH_DAY_OF_WEEK,DAMAGE,VEHICLE_TYPE,BAC_RESULT,...,LIGHTING_CONDITION,WEATHER_CONDITION,AIRBAG_DEPLOYED,ROAD_DEFECT,PRIM_CONTRIBUTORY_CAUSE,SEC_CONTRIBUTORY_CAUSE,FIRST_CONTACT_POINT,POSTED_SPEED_LIMIT,CRASH_HOUR,target_mapped
12,34.0,NO INDICATION OF INJURY,TOYOTA,CAMRY,2001.0,02/23/2020 06:15:00 PM,1,"OVER $1,500",PASSENGER,TEST NOT OFFERED,...,DUSK,CLEAR,DID NOT DEPLOY,NO DEFECTS,IMPROPER OVERTAKING/PASSING,UNABLE TO DETERMINE,FRONT-LEFT-CORNER,30,18,reckless driving
18,22.0,NO INDICATION OF INJURY,TOYOTA,AVALON,2006.0,05/06/2020 12:53:00 AM,4,"$501 - $1,500",PASSENGER,TEST NOT OFFERED,...,"DARKNESS, LIGHTED ROAD",CLEAR,DID NOT DEPLOY,NO DEFECTS,IMPROPER LANE USAGE,UNABLE TO DETERMINE,FRONT-RIGHT-CORNER,15,0,reckless driving
37,22.0,NO INDICATION OF INJURY,CHEVROLET,UNKNOWN,2013.0,02/23/2020 06:22:00 PM,1,"OVER $1,500",PASSENGER,TEST NOT OFFERED,...,DARKNESS,CLEAR,"DEPLOYED, FRONT",UNKNOWN,IMPROPER LANE USAGE,NOT APPLICABLE,SIDE-RIGHT-FRONT,30,18,reckless driving
38,61.0,NO INDICATION OF INJURY,CHEVROLET,SILVERADO,2017.0,05/02/2020 11:00:00 AM,7,"OVER $1,500",SPORT UTILITY VEHICLE (SUV),TEST NOT OFFERED,...,DAYLIGHT,CLEAR,DID NOT DEPLOY,NO DEFECTS,FOLLOWING TOO CLOSELY,NOT APPLICABLE,REAR,45,11,reckless driving
42,28.0,NO INDICATION OF INJURY,HONDA,CR-V,2002.0,03/25/2022 05:10:00 AM,6,"OVER $1,500",PASSENGER,TEST NOT OFFERED,...,"DARKNESS, LIGHTED ROAD",CLEAR,"DEPLOYED, COMBINATION",NO DEFECTS,FAILING TO YIELD RIGHT-OF-WAY,NOT APPLICABLE,FRONT-LEFT-CORNER,35,5,missed or misunderstood traffic signs
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
731887,33.0,NO INDICATION OF INJURY,JEEP,CHEROKEE,1998.0,04/18/2023 05:30:00 AM,3,"$501 - $1,500",SPORT UTILITY VEHICLE (SUV),TEST NOT OFFERED,...,"DARKNESS, LIGHTED ROAD",CLEAR,"DEPLOYED, COMBINATION",NO DEFECTS,ROAD CONSTRUCTION/MAINTENANCE,NOT APPLICABLE,FRONT-LEFT-CORNER,40,5,external variable
731891,21.0,NO INDICATION OF INJURY,HONDA,ACCORD,2005.0,04/18/2023 07:00:00 PM,3,"OVER $1,500",PASSENGER,TEST NOT OFFERED,...,DUSK,CLEAR,DID NOT DEPLOY,NO DEFECTS,FAILING TO YIELD RIGHT-OF-WAY,UNABLE TO DETERMINE,FRONT-LEFT-CORNER,30,19,missed or misunderstood traffic signs
731899,44.0,NO INDICATION OF INJURY,HONDA,ODYSSEY,2014.0,04/18/2023 05:00:00 PM,3,"$501 - $1,500",SPORT UTILITY VEHICLE (SUV),TEST NOT OFFERED,...,DAYLIGHT,CLEAR,DID NOT DEPLOY,NO DEFECTS,IMPROPER LANE USAGE,NOT APPLICABLE,SIDE-LEFT,30,17,reckless driving
731902,23.0,"REPORTED, NOT EVIDENT",JEEP,PATRIOT,2015.0,04/18/2023 11:32:00 AM,3,"OVER $1,500",PASSENGER,TEST NOT OFFERED,...,DAYLIGHT,CLEAR,"DEPLOYED, COMBINATION",NO DEFECTS,TURNING RIGHT ON RED,NOT APPLICABLE,SIDE-RIGHT,30,11,reckless driving


# Now lets go back to modeling

In [522]:
numeric_pipeline = Pipeline([('numnorm', StandardScaler())])

#ordinal_pipeline = Pipeline([
  #  ('ordenc', OrdinalEncoder(categories = [columns])),
  #  ('ordnorm', StandardScaler())])

nominal_pipeline = Pipeline([
    ('onehotenc', OneHotEncoder(sparse = False, handle_unknown = 'ignore')), 
    ('onehotnorm', MaxAbsScaler())])

In [523]:
cat_cols = ['MAKE', 'MOST_SEVERE_INJURY', 'DAMAGE', 'VEHICLE_TYPE', 'LIGHTING_CONDITION', 
            'AIRBAG_DEPLOYED', 'BAC_RESULT', 'ROAD_DEFECT', 'FIRST_CONTACT_POINT',
           'SAFETY_EQUIPMENT', 'WEATHER_CONDITION']
num_cols = ['AGE', 'VEHICLE_YEAR', 'CRASH_HOUR', 'CRASH_DAY_OF_WEEK', 'POSTED_SPEED_LIMIT']

In [524]:
ct = ColumnTransformer(
    [ ("nominalpipe", nominal_pipeline, cat_cols),
      ("numericpipe", numeric_pipeline, num_cols)
    ])

In [525]:
pipe = Pipeline([('preprocess', ct), 
                     ('model',
                       LogisticRegression(max_iter = 200000))])
pipe

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('nominalpipe',
                                                  Pipeline(steps=[('onehotenc',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False)),
                                                                  ('onehotnorm',
                                                                   MaxAbsScaler())]),
                                                  ['MAKE', 'MOST_SEVERE_INJURY',
                                                   'DAMAGE', 'VEHICLE_TYPE',
                                                   'LIGHTING_CONDITION',
                                                   'AIRBAG_DEPLOYED',
                                                   'BAC_RESULT', 'ROAD_DEFECT',
                                                   'FIRST_CONTACT_POINT',

In [526]:
df3 = df2[['MAKE', 'MOST_SEVERE_INJURY', 'DAMAGE', 'VEHICLE_TYPE', 'LIGHTING_CONDITION', 
            'AIRBAG_DEPLOYED', 'BAC_RESULT', 'ROAD_DEFECT', 'FIRST_CONTACT_POINT',
           'SAFETY_EQUIPMENT', 'WEATHER_CONDITION', 'AGE', 'VEHICLE_YEAR', 'CRASH_HOUR', 
           'CRASH_DAY_OF_WEEK', 'target_mapped', 'POSTED_SPEED_LIMIT']]

In [527]:
X = df3.drop('target_mapped', axis=1)
y = df3['target_mapped']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

pipe.fit(X_train, y_train)

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('nominalpipe',
                                                  Pipeline(steps=[('onehotenc',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False)),
                                                                  ('onehotnorm',
                                                                   MaxAbsScaler())]),
                                                  ['MAKE', 'MOST_SEVERE_INJURY',
                                                   'DAMAGE', 'VEHICLE_TYPE',
                                                   'LIGHTING_CONDITION',
                                                   'AIRBAG_DEPLOYED',
                                                   'BAC_RESULT', 'ROAD_DEFECT',
                                                   'FIRST_CONTACT_POINT',

In [528]:
print(pipe.score(X_train,y_train))
pipe.score(X_test,y_test)

0.593038829242797


0.5935016006446133

In [529]:
pipe1 = Pipeline([('preprocess', ct), 
                 ('model', XGBClassifier(eval_metric='logloss'))])

pipe1.fit(X_train, y_train)


Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('nominalpipe',
                                                  Pipeline(steps=[('onehotenc',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False)),
                                                                  ('onehotnorm',
                                                                   MaxAbsScaler())]),
                                                  ['MAKE', 'MOST_SEVERE_INJURY',
                                                   'DAMAGE', 'VEHICLE_TYPE',
                                                   'LIGHTING_CONDITION',
                                                   'AIRBAG_DEPLOYED',
                                                   'BAC_RESULT', 'ROAD_DEFECT',
                                                   'FIRST_CONTACT_POINT',

In [511]:
# from sklearn.model_selection import GridSearchCV

# param_grid = {
#     'model__learning_rate': [0.01, 0.1, 0.2],
#     'model__max_depth': [3, 5, 7],
#     'model__min_child_weight': [1, 3],
#     'model__subsample': [0.5, 0.7, 1],
#     'model__colsample_bytree': [0.5, 0.7, 1]
# }

# grid_search = GridSearchCV(pipe1, param_grid, cv=3, scoring='accuracy')
# grid_search.fit(X_train, y_train)

# print("Best parameters: ", grid_search.best_params_)
# print("Best score: ", grid_search.best_score_)


Best parameters:  {'model__colsample_bytree': 1, 'model__learning_rate': 0.01, 'model__max_depth': 5, 'model__min_child_weight': 3, 'model__subsample': 0.5}
Best score:  0.566867739246008


In [530]:
print(pipe1.score(X_train,y_train))
pipe1.score(X_test,y_test)

0.6232278577495155


0.6013632701060563

In [484]:
# Get the OneHotEncoder from the pipeline
one_hot_encoder = pipe.named_steps['preprocess'].named_transformers_['nominalpipe'].named_steps['onehotenc']

# Get the categorical feature names
categorical_feature_names = one_hot_encoder.get_feature_names(input_features=cat_cols)

# Combine the categorical and numerical feature names
all_feature_names = np.concatenate([categorical_feature_names, num_cols])

# Now compute the feature importances
feature_importance = pipe.named_steps['model'].coef_[0]
importances = pd.Series(feature_importance, index=all_feature_names)
importances = importances.abs().sort_values(ascending=False)
importances

WEATHER_CONDITION_SNOW                                      0.299750
WEATHER_CONDITION_CLEAR                                     0.261509
ROAD_DEFECT_RUT, HOLES                                      0.244866
WEATHER_CONDITION_RAIN                                      0.167165
ROAD_DEFECT_OTHER                                           0.126833
BAC_RESULT_TEST TAKEN                                       0.111495
SAFETY_EQUIPMENT_SAFETY BELT USED                           0.110868
SAFETY_EQUIPMENT_USAGE UNKNOWN                              0.107653
FIRST_CONTACT_POINT_UNDER CARRIAGE                          0.089063
ROAD_DEFECT_NO DEFECTS                                      0.087590
FIRST_CONTACT_POINT_SIDE-RIGHT                              0.082671
BAC_RESULT_TEST NOT OFFERED                                 0.082564
WEATHER_CONDITION_SLEET/HAIL                                0.071907
WEATHER_CONDITION_CLOUDY/OVERCAST                           0.069236
CRASH_HOUR                        